---

Tutorial na podstawie:
- [1] Julia High Performance Programming - Ivo Balbaert
- [2] http://docs.julialang.org/en/stable/manual/performance-tips/
- [3] Wykład 1

Zawartosc:

- type-stability ([1] str. 256)
- wydajność typów złożonych prostych i abstrakcyjnych
- rozbijanie funkcji na miejsze
- bariery funkcji

Przydatne materiały z pierwszego wykładu:

- makra: @inbounds i @inbounds, więcej w [1] (str. 313)
- zmienne globalne, więcej w [1] (str. 277)

---

In [14]:
versioninfo()
Pkg.clone("https://github.com/johnmyleswhite/Benchmarks.jl")
Pkg.add("Benchmarks")
using Benchmarks

Julia Version 0.5.0
Commit 3c9d753 (2016-09-19 18:14 UTC)
Platform Info:
  System: Linux (x86_64-linux-gnu)
  CPU: Westmere E56xx/L56xx/X56xx (Nehalem-C)
  WORD_SIZE: 64
  BLAS: libopenblas (NO_LAPACKE DYNAMIC_ARCH NO_AFFINITY Nehalem)
  LAPACK: liblapack.so.3
  LIBM: libopenlibm
  LLVM: libLLVM-3.7.1 (ORCJIT, westmere)


INFO: Cloning Benchmarks from https://github.com/johnmyleswhite/Benchmarks.jl


LoadError: Benchmarks already exists

## Type-stability
---

In [15]:
# Type unstable version
@benchmark trunc(2.5)

================ Benchmark Results ========================
     Time per evaluation: 3.31 ns [3.26 ns, 3.35 ns]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 8501
   Number of evaluations: 6600001
         R² of OLS model: 0.955
 Time spent benchmarking: 8.82 s


In [19]:
# Type stable version

function trunc_fixed(x)
    if x < 0
        return zero(x)
    else
        return x
    end
end

@benchmark trunc_fixed(2.5)

================ Benchmark Results ========================
     Time per evaluation: 2.57 ns [2.51 ns, 2.62 ns]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 3901
   Number of evaluations: 82201
         R² of OLS model: 0.955
 Time spent benchmarking: 3.53 s


In [20]:
# to warun against type unstability
@code_warntype trunc(2.5)

Variables:
  #self#::Base.#trunc
  x::Float64

Body:
  begin 
      return (Base.box)(Base.Float64,(Base.trunc_llvm)(x::Float64))
  end::Float64


## Typy złożone

Wydajność typów złożonych. Brak informacji o type, uniemożliwia kompilatorowi podjęcie decyzji o konieczności używani *pointer inderection*. Więcej w [1] str. 273.

In [21]:
# Bad Performacne
immutable BadPoint
    x
    y
end

# Good Performance
immutable GoodPoint
    x::Float64
    y::Float64
end

LoadError: invalid redefinition of constant Point


### Typy abstrakcyjne

Uzywając drugiego sposobu, kompilator może dokonać optymalizacji. [1] str. 274

In [22]:
# Bad Performacne
immutable AbstractBadPoint
    x::AbstractFloat
    y::AbstractFloat
end

# Good Performance
immutable AbstractGoodPoint{ T <: AbstractFloat}
    x::T
    y::T
end

## Dzielenie funkcji ma mniejsze

Praktyka ta pomaga w kompilatorowi w inlinowaniu kodu [1] (str. 282) i rzecz jasna zmniejsza ilość operacji np. typu flow control [2].

In [23]:
# Zamiast
function norm(A)
    if isa(A, Vector)
        return sqrt(real(dot(A,A)))
    elseif isa(A, Matrix)
        return max(svd(A)[2])
    else
        error("norm: invalid argument")
    end
end

# Piszemy
norm(x::Vector) = sqrt(real(dot(x,x)))
norm(A::Matrix) = max(svd(A)[2])

norm (generic function with 3 methods)

## Bariery Funkcji

Kolejna optymalizacja mająca pomóc kompilatorowi w rozpoznawaniu typów. W przypadki drugi, kompilator może wygenerować kilka typów konkretnych funkcji dla różnych typów wywpłania, zamiast trzymać jedną z wywpłaniem o niezdefiniowanym typie. Więcej na [2].

In [44]:
# Zamiast
function strange_twos(n)
    a = Array(rand(Bool) ? Int64 : Float64, n)
    for i = 1:n
        a[i] = 2
    end
    return a
end

# lepiej napisać
function fill_twos!(a)
    for i=1:length(a)
        a[i] = 2
    end
end

function strange_twos(n)
    a = Array(rand(Bool) ? Int64 : Float64, n)
    fill_twos!(a)
    return a
end